<font size = 3>**Compare results across classifiers and parameters**

**XGB** with or without second classifier (always XGB) <br>
**CNN** with or without second classifier (always XGB) <br><br>
<i>Both across various optical flow parameters</i>

In [2]:
import sys
sys.path.append("/users/tom/git/neon_blink_detection/")
sys.path.append("/users/tom/git/neon_blink_detection/src")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import numpy as np

cnn_path = "/users/tom/git/neon_blink_detection/training-XGBClassifier-3-200320231657/n_lay5-lay_intv7-grid4-win15-trans0.0-scale0.0-speed0.0/results.pkl"

with open(cnn_path, 'rb') as f:
    cnn_results = pickle.load(f)

xgb_path = "/users/tom/git/neon_blink_detection/training-XGBClassifier-3-100320231148/n_lay5-lay_intv7-grid4-win15-trans0.0-scale0.0/results.pkl"

with open(xgb_path, 'rb') as f:
    xgb_results = pickle.load(f)

In [33]:
precision = cnn_results.metrics_sample_val



# fig, ax = plt.subplots()

mean_precision_cnn = np.mean([cnn_results.metrics_pp_val.scores_list[x].F1 for x in range(5)])
mean_recall_cnn = np.mean([cnn_results.metrics_pp_val.scores_list[x].recall for x in range(5)])
mean_f1_score_cnn = np.mean([cnn_results.metrics_pp_val.scores_list[x].precision for x in range(5)])

mean_precision_xgb = np.mean([xgb_results.metrics_pp_val.scores_list[x].F1 for x in range(5)])
mean_recall_xgb = np.mean([xgb_results.metrics_pp_val.scores_list[x].recall for x in range(5)])
mean_f1_score_xgb = np.mean([xgb_results.metrics_pp_val.scores_list[x].precision for x in range(5)])



In [35]:
# print results in a visally appealing way
print("CNN")
print("--------------------")
print("Precision: ", mean_precision_cnn)
print("Recall: ", mean_recall_cnn)
print("F1 Score: ", mean_f1_score_cnn)
print("XGB")
print("--------------------")
print("Precision: ", mean_precision_xgb)
print("Recall: ", mean_recall_xgb)
print("F1 Score: ", mean_f1_score_xgb)

CNN
--------------------
Precision:  0.9487285807744603
Recall:  0.9711876729965162
F1 Score:  0.9275466876311128
XGB
--------------------
Precision:  0.9338402586407414
Recall:  0.960949825028932
F1 Score:  0.9087036289905799


In [36]:

mean_precision_cnn = np.mean([cnn_results.metrics_pp_test.scores_list[x].F1 for x in range(5)])
mean_recall_cnn = np.mean([cnn_results.metrics_pp_test.scores_list[x].recall for x in range(5)])
mean_f1_score_cnn = np.mean([cnn_results.metrics_pp_test.scores_list[x].precision for x in range(5)])

mean_precision_xgb = np.mean([xgb_results.metrics_pp_test.scores_list[x].F1 for x in range(5)])
mean_recall_xgb = np.mean([xgb_results.metrics_pp_test.scores_list[x].recall for x in range(5)])
mean_f1_score_xgb = np.mean([xgb_results.metrics_pp_test.scores_list[x].precision for x in range(5)])

In [37]:
# print results in a visally appealing way
print("CNN")
print("--------------------")
print("Precision: ", mean_precision_cnn)
print("Recall: ", mean_recall_cnn)
print("F1 Score: ", mean_f1_score_cnn)
print("XGB")
print("--------------------")
print("Precision: ", mean_precision_xgb)
print("Recall: ", mean_recall_xgb)
print("F1 Score: ", mean_f1_score_xgb)

CNN
--------------------
Precision:  0.961206568381202
Recall:  0.9772771792360431
F1 Score:  0.9457606414262774
XGB
--------------------
Precision:  0.9603155706528002
Recall:  0.9808031341821742
F1 Score:  0.9407025566034226


In [32]:
mean_f1_score_xgb

0.9407025566034226

In [21]:
cnn_results.metrics_pp_val.n_samples_list

[1233074, 454176, 300879, 104046, 189164, 159913]

In [8]:
dir(cnn_results.metrics_pp_val.scores_list[0])

['F1',
 'FN',
 'FP',
 'IoU',
 'RTD_offset',
 'RTD_onset',
 'RTO_offset',
 'RTO_onset',
 'TP',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'confusion_matrix',
 'deletions',
 'duration_gt',
 'duration_pd',
 'insertions',
 'mean_IoU',
 'precision',
 'recall',
 'replace']

In [3]:
import sys

sys.path.append("/users/tom/git/neon_blink_detection/")
sys.path.append("/users/tom/git/neon_blink_detection/src")

from training.evaluation import evaluate
from training.helper import ClassifierParams, Results
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import cross_validate
from copy import copy
from functions.classifiers import Classifier, load_predictions, save_predictions
from src.post_processing import classify
from src.neon_blink_detector import get_params
of_params, pp_params, _ = get_params()
from src.features_calculator import create_grids
from pathlib import Path
import numpy as np
from random import choices
from training.datasets_loader import (
    concatenate,
    concatenate_all_samples,
)
from training.helper import (
    get_augmentation_options,
    get_export_dir,
    get_training_dir,
)
from src.event_array import Samples
from src.helper import OfParams, PPParams, AugParams
from training.helper import get_experiment_name_new
from training.video_loader import video_loader

clip_names = np.load("/users/tom/git/neon_blink_detection/clip_list.npy")

of_params = OfParams(n_layers=5, layer_interval=7, average=False, img_shape=(64, 64), grid_size=20, step_size=7, window_size=11, stop_steps=3)

aug_params_options = get_augmentation_options()
aug_params = aug_params_options[0]

rec = video_loader(of_params, aug_params)

rec.collect(clip_names, bg_ratio=1, augment=False, idx=0)

options 5

Loading clip: 1316-2023-02-03-09-34-09-31ef7938
Number of clips: 4

Loading clip: 1311-2023-02-02-14-30-40-96be0928
Number of clips: 1

Loading clip: 1312-2023-02-02-16-21-47-cd28e8e0
Number of clips: 1

Loading clip: 1313-2023-02-02-16-31-53-67b1bdbe
Number of clips: 1

Loading clip: 1329-2023-02-06-10-40-29-c4308424
Number of clips: 2

Loading clip: 1332-2023-02-06-11-58-03-8fc75e25
Number of clips: 1

Loading clip: 1333-2023-02-06-13-17-50-ebc1e021
Number of clips: 1

Loading clip: 1336-2023-02-06-15-28-30-adb9dbfb
Number of clips: 2

Loading clip: 1337-2023-02-06-15-36-31-211278ea
Number of clips: 1

Loading clip: 1338-2023-02-06-16-29-00-adceb23d
Number of clips: 1

Loading clip: 1339-2023-02-06-17-24-49-5b24f5ba
Number of clips: 1

Loading clip: 1340-2023-02-06-17-37-00-e6218648
Number of clips: 1

Loading clip: 1341-2023-02-07-10-38-42-d828e5ac
Number of clips: 4

Loading clip: 1342-2023-02-07-10-43-55-b5709e1f
Number of clips: 2

Loading clip: 1343-2023-02-07-11-37-0

In [20]:
import joblib
from src.post_processing import post_process
from functions.classifiers import Classifier
from functions.pipeline import get_classifier_params

features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels

xgb_path = "/users/tom/git/neon_blink_detection/training-XGBClassifier-3-100320231148/n_lay5-lay_intv7-grid4-win15-trans0.0-scale0.0/results.pkl"

clf_path = "/users/tom/git/neon_blink_detection/weights/xgb.sav"
clf = joblib.load(str(clf_path))

classifier_params = get_classifier_params(type="XGB", use_second_classifier=False)
clf_path = "/users/tom/git/neon_blink_detection/weights/"
clf = Classifier(classifier_params, clf_path)
clf.load_base_classifier(idx=0)

In [24]:
clf.predict_all_clips(rec.all_features)


ValueError: Feature shape mismatch, expected: 160, got 4000

(3752, 4000)